In [7]:
path_to_home = "../"
#path_to_home = "./drive/MyDrive/receiptlayoutlm/"

In [2]:
from huggingface_hub import Repository
hf_repo = Repository(path_to_home + "hugging_face_repo")

In [ ]:
hf_repo.git_pull()

In [8]:
from transformers import LayoutLMv2FeatureExtractor, LayoutXLMTokenizer, LayoutXLMProcessor
from transformers import LayoutLMv2ForTokenClassification
from transformers import pipeline

feature_extractor = LayoutLMv2FeatureExtractor()
tokenizer_xlm = LayoutXLMTokenizer.from_pretrained("microsoft/layoutxlm-base")
processor_xlm = LayoutXLMProcessor(feature_extractor, tokenizer_xlm)
model = LayoutLMv2ForTokenClassification.from_pretrained("sibrun/receiptlayoutlm", use_auth_token=True)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LayoutLMv2Tokenizer'. 
The class this function is called from is 'LayoutXLMTokenizer'.


In [9]:
from PIL import Image
import numpy as np
import torch

path_to_image = "../data/X00016469612.jpg"
image = Image.open(path_to_image)


In [38]:
features = feature_extractor(image, return_tensors="pt")
tokenizer_output = tokenizer_xlm(
        text=features['words'],
        boxes=features['boxes'],
        is_split_into_words=True,
        truncation=True,
        return_tensors='pt')
model_input = tokenizer_output
model_input['image'] = features['pixel_values']
with torch.no_grad():
        model_output = model(**model_input)
logits = model_output.logits.numpy()[0]
predictions = np.argmax(logits, axis=-1).tolist()
ner_labels = [model.config.id2label[p] for p in predictions]
tokens = [tokenizer_xlm.convert_ids_to_tokens(id) for id in tokenizer_output['input_ids'][0].tolist()]

/Users/simon/PycharmProjects/receiptlayoutlm/venv/lib/python3.9/site-packages/transformers/models/layoutlmv2/modeling_layoutlmv2.py:772: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  torch.arange(
/Users/simon/PycharmProjects/receiptlayoutlm/venv/lib/python3.9/site-packages/transformers/models/layoutlmv2/modeling_layoutlmv2.py:782: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, us

In [43]:
import pandas as pd
pd.DataFrame([tokens, ner_labels], index=["tokens", "tags"])

,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
tokens,<s>,▁tan,▁woon,▁yan,n,▁,BOOK,▁TA,▁-,K,...,K,▁YOU,▁,PLE,ASE,▁COME,▁AG,AIN,▁t,</s>
tags,O,O,O,O,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O
